In [1]:
# Imports and data loading
import pandas as pd

first_150k = pd.read_csv('../input/wine-reviews/winemag-data_first150k.csv', index_col=0)
data_csv = pd.read_csv('../input/wine-reviews/winemag-data-130k-v2.csv', index_col=0)
data_json = pd.read_json('../input/wine-reviews/winemag-data-130k-v2.json')

data = data_csv
target_data = data['points']
feature_data = data.drop('points', axis=1)

## Read the manual
The 'manual' for this data comes in the form of text descriptions on the kaggle site about this data that I have copied and pasted into a txt file, then formatted. 

>Here is a CSV version of the data I scraped. This dataset has three new fields  
--Title (which you can parse the vintage from), Taster Name, and Taster Twitter Handle. This should also fix the duplicate entries problem in the first version of the dataset and add ~25k unique reviews to play with. 

So There are two copies of the dataset, one of which I can see is duplicated in .json form. 

> After watching Somm (a documentary on master sommeliers) I wondered how I could create a predictive model to identify wines through blind tasting like a master sommelier would. The first step in this journey was gathering some data to train a model. I plan to use deep learning to predict the wine variety using words in the description/review. The model still won't be able to taste the wine, but theoretically it could identify the wine based on a description that a sommelier could give. If anyone has any ideas on how to accomplish this, please post them!

So their purpose was similar to mine: pick good wines. 

>winemag-data-130k-v2.csv contains 10 columns and 130k rows of wine reviews.  
winemag-data_first150k.csv contains 10 columns and 150k rows of wine reviews.  
winemag-data-130k-v2.json contains 6919 nodes of wine reviews.

There are three files, one of which is the 'first', and the other two are 'v2', but in different file formats. 

>The data was scraped from WineEnthusiast during the week of June 15th, 2017. The code for the scraper can be found here if you have any more specific questions about data collection that I didn't address.  
**UPDATE 11/24/2017** After feedback from users of the dataset I scraped the reviews again on November 22nd, 2017. This time around I collected the title of each review, which you can parse the year out of, the tasters name, and the taster's Twitter handle. This should also fix the duplicate entry issue.

The data was acquired by web scraping, and then an updated version was scraped that removed some issues with the data. I guess that's what 'first' and 'v2' are referring to. 

>**Inspiration**  
I think that this dataset offers some great opportunities for sentiment analysis and other text related predictive models. My overall goal is to create a model that can identify the variety, winery, and location of a wine based on a description. If anyone has any ideas, breakthroughs, or other interesting insights/models please post them.

Yes. This is a worthy goal. 

>Numeric: unnamed, but functions as a count from 0 ID number  
**String: country**  
The country that the wine is from  
**String: description**  
A few sentences from a sommelier describing the wine's taste, smell, look, feel, etc.  
**String: designation**  
The vineyard within the winery where the grapes that made the wine are from  
**Numeric: points**  
The number of points WineEnthusiast rated the wine on a scale of 1-100 (though they say they only post reviews for wines that score >=80)  
**Numeric: price**  
The cost for a bottle of the wine  
**String: province**  
The province or state that the wine is from  
**String: region_1**  
The wine growing area in a province or state (ie Napa)  
**String: region_2**  
Sometimes there are more specific regions specified within a wine growing area (ie Rutherford inside the Napa Valley), but this value can sometimes be blank  
**String: taster_name**  
Name of the person who tasted and reviewed the wine  
**String: taster_twitter_handle**  
Twitter handle for the person who tasted and reviewed the wine  
**String: title**  
The title of the wine review, which often contains the vintage if you're interested in extracting that feature  
**String: variety**  
The type of grapes used to make the wine (ie Pinot Noir)  
**String: winery**  
The winery that made the wine  

Useful data types and descriptions for the 14 columns. One of them is just an ID column, which I'll incorporate into the dataframe upon loading. The rest are useful looking things, including a freeform text description, name, point score, price, location data, reviewer data (who wrote the description and gave the point score), some text data indicating it's name and year the grapes were picked, the variety of grapes, and the winery that produced the wine. 

Most significantly, above all previous wine-related datasets, these features (besides points and reviewer) are things I can actually find out by reading the label in the shop. This is probably the most useful wine related dataset so far (it's also by far the largest). 

The text data is all freeform and needs to be processed to make it usable. Some needs simple encoding, others require further processing to be useful. 

## Basics
What does the data look like?  
How much of it is there?  
What type of data is it? 

In [2]:
first_150k.sample(5, random_state=4)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
31034,US,"The aromas seem a bit muddled, with notes of j...",Colter's Creek Estate,84,10.0,Idaho,Idaho,NaN,Riesling,Colter's Creek
58,US,Cabernet makes up just over half of this blend...,Final Final,90,30.0,Washington,Columbia Valley (WA),Columbia Valley,Cabernet Sauvignon-Syrah,Efeste
24657,Italy,"Le Macchiole's Bolgheri Rosso blend (Merlot, C...",NaN,90,30.0,Tuscany,Bolgheri,NaN,Red Blend,Le Macchiole
139801,Italy,"A special, gentle wine with caressing persiste...",NaN,92,60.0,Tuscany,Brunello di Montalcino,NaN,Sangiovese,Fornacina
1820,US,Winemaker Nick Goldschmidt does wonders with t...,Lone Tree Vineyard,90,50.0,California,Alexander Valley,Sonoma,Cabernet Sauvignon,Forefathers


The first data collected has fewer columns, but is otherwise as described. Since I've already decided not to use the old data, and to work with the more recently collected data (that also has more features), I will not analyse this any further. 

In [3]:
data_csv.sample(5, random_state=4)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
105424,France,Steinbach is a village at the extreme southern...,Steinbach,90,35.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Joseph Cattin 2010 Steinbach Pinot Noir (Alsace),Pinot Noir,Joseph Cattin
30811,US,"Salted lemon rinds, ice-cold apple, fresh-sque...",Bien Nacido Vineyard Block II Mount Eden Clone,92,46.0,California,Santa Maria Valley,Central Coast,Matt Kettmann,@mattkettmann,Scar of the Sea 2013 Bien Nacido Vineyard Bloc...,Chardonnay,Scar of the Sea
86983,Italy,"Delicately scented, this offers subtle aromas ...",Occhi di Ciumi,89,20.0,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Al-Cantàra 2015 Occhi di Ciumi (Etna),White Blend,Al-Cantàra
109619,US,"As sweet as angel food cake, with sugary vanil...",NaN,82,14.0,California,Central Coast,Central Coast,NaN,NaN,Cupcake 2008 Chardonnay (Central Coast),Chardonnay,Cupcake
98442,France,"Very dense and firm, this has powerful tannins...",NaN,94,NaN,Bordeaux,Pessac-Léognan,NaN,Roger Voss,@vossroger,Château Smith Haut Lafitte 2009 Pessac-Léognan,Bordeaux-style Red Blend,Château Smith Haut Lafitte


The features all seem to be as described. Notably, the points are in integers and price is in decimals. Also, in this random sample of 5 rows I can already see seven data are missing. Much of the longer text data is inconsistent in structure, though all seem to have a year (vintage) which - if present in enough of them - could be useful for determining the age of the wine. Relevantly, the reviews were scraped on 22/02/2017. 

In [4]:
data_json.sample(4, random_state=4)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
105424,France,Steinbach is a village at the extreme southern...,Steinbach,90,35.0,Alsace,Alsace,None,Roger Voss,@vossroger,Joseph Cattin 2010 Steinbach Pinot Noir (Alsace),Pinot Noir,Joseph Cattin
30811,US,"Salted lemon rinds, ice-cold apple, fresh-sque...",Bien Nacido Vineyard Block II Mount Eden Clone,92,46.0,California,Santa Maria Valley,Central Coast,Matt Kettmann,@mattkettmann,Scar of the Sea 2013 Bien Nacido Vineyard Bloc...,Chardonnay,Scar of the Sea
86983,Italy,"Delicately scented, this offers subtle aromas ...",Occhi di Ciumi,89,20.0,Sicily & Sardinia,Etna,None,Kerin O’Keefe,@kerinokeefe,Al-Cantàra 2015 Occhi di Ciumi (Etna),White Blend,Al-Cantàra
109619,US,"As sweet as angel food cake, with sugary vanil...",None,82,14.0,California,Central Coast,Central Coast,None,None,Cupcake 2008 Chardonnay (Central Coast),Chardonnay,Cupcake


Immediately, this data looks the same as `data_csv`. 

In [5]:
data_json.equals(data_csv)

True

They are exactly the same. I can safely disregard `data_json`. This leaves a single data file, in CSV format, left to continue with. 

In [6]:
data_csv.shape

(129971, 13)

With 13 features and 129971 samples, there are 1,689,623 pices of data. 
- **country**: String. Nominal categorical. 
- **description**: String. Freeform text. 
- **designation**: String. Freeform text. 
- **points**: Numeric. Integer ratio. 
- **price**: Numeric: Continuous ratio. 
- **province**: String. Nominal categorical. 
- **region_1**: String. Nominal categorical. 
- **region_2**: String. Nominal categorical. 
- **taster_name**: String. Nominal categorical. 
- **taster_twitter_handle**: String. Nominal categorical. 
- **title**: String. Freeform text. Contains an integer interval datum that could be parsed. 
- **variety**: String. Nominal categorical. 
- **winery**: String. Nominal categorical. 

## Target variable
Datatype?  
Descriptive statistics?  
Unbalanced? Does it need weighting?  
Is it linearly separable?  
What sorts of models can predict this?  

The target variable is `points`. It's data type is integer ratio. 

In [7]:
# Descriptive statistics
target_data.describe()

count    129971.000000
mean         88.447138
std           3.039730
min          80.000000
25%          86.000000
50%          88.000000
75%          91.000000
max         100.000000
Name: points, dtype: float64

There is no missing data in the target variable. There are also no scores below 80, which on a 0-100 score system is farcical. 

Having just read the code that scraped the website, and then to the website itself, I've found that it was hardcoded to scrape all the pages of reviews that were there at the time. This calls into question the reliability of this dataset. 